In [ ]:
from pyroclastmpm import (
    NoSlipWall,
    SlipWall,
    Gravity,
    NewtonFluid,
    ParticlesContainer,
    NodesContainer,
    USL,
    MUSL,
    APIC,
    CubicShapeFunction,
    set_globals,
)

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Define simulation parameters

L = 1.2  # m
H = 0.8
cs = 1.0 / 40

rho0 = 1800
K = 0.36 * 1000
G = 0.01

c = np.sqrt(K / rho0)

ppc = 2

# Define simulation parameters
set_globals(
    dimension=3,
    dt=5e-5,
    shape_function=CubicShapeFunction,
    output_directory="./output",
)

In [ ]:
nodes = NodesContainer(
    node_start=np.array([0.0, 0.0, 0.0]),
    node_end=np.array([L + 4 * cs, H + 4 * cs, L + 4 * cs]),
    node_spacing=cs,
)


node_coords = nodes.give_coords()


wspread = np.arange(0, 0.6 + cs, cs / ppc) + 6.5 * (cs)
hspread = np.arange(0, 0.6 + cs, cs / ppc) + 4.5 * (cs)
lspread = np.arange(0, 0.6 + cs, cs / ppc) + 6.5 * (cs)
xv, yv, zv = np.meshgrid(wspread, hspread, lspread)

particle_coords = np.array(
    list(zip(xv.flatten(), yv.flatten(), zv.flatten()))
).astype(np.float64)
print(f"num_p {len(particle_coords)}, num_c {nodes.num_nodes_total} \n")

In [ ]:
vels = np.zeros(particle_coords.shape)

mass = np.ones(len(particle_coords)) * rho0 * cs * cs

volume = np.ones(len(particle_coords)) * cs

particles = ParticlesContainer(
    positions=particle_coords, masses=mass, volumes=volume, velocities=vels
)


material = NewtonFluid(viscocity=G, bulk_modulus=K, gamma=7)

particles.materials = [material]

wally0 = NoSlipWall(wallplane="y0")
wally1 = NoSlipWall(wallplane="y1")

wallx0 = SlipWall(wallplane="x0")
wallx1 = SlipWall(wallplane="x1")

wallz0 = SlipWall(wallplane="z0")
wallz1 = SlipWall(wallplane="z1")


wallgrav = Gravity(gravity=np.array([0, -40.81, 0]))


MPM = APIC(
    particles=particles,
    nodes=nodes,
    total_steps=50000,
    output_steps=500,
    output_start=0,
    boundaryconditions=[
        wally0,
        wally1,
        wallx0,
        wallx1,
        wallz0,
        wallz1,
        wallgrav,
    ],
)

In [ ]:
MPM.run()